Here we use the information of the VDJdb V1 stored in W&B with the complexID: 2178

In [1]:
from Stitchr import stitchrfunctions as fxn
from Stitchr import stitchr as st

This is the official documentatin if one would like to use Stitchr in an individual pipeline: 
[Stitchr Doc](https://github.com/JamieHeather/stitchr/blob/main/docs/importing.rst)

In [2]:
chain = "TRB"
species = "HUMAN"

tcr_dat, functionality, partial = fxn.get_imgt_data(chain, st.gene_types, species)
codons = fxn.get_optimal_codons('', species)

V_segment = "TRBV9*01"
J_segment = "TRBJ2-2*01"
CDR3_sequence_beta = "CASSENTANTGELFF"

tcr_bits = {'v': V_segment, 'j': J_segment, 'cdr3': CDR3_sequence_beta,
            'l': 'TRBV7-3*01', 'c': 'TRBC1*01',
            'skip_c_checks': False, 'species': species, 'seamless': False,
            '5_prime_seq': '', '3_prime_seq': '', 'name': 'TCR'}

stitched = st.stitch(tcr_bits, tcr_dat, functionality, partial, codons, 3, '')
stitched

(['TCR',
  'TRBV9*01',
  'TRBJ2-2*01',
  'TRBC1*01',
  'CASSENTANTGELFF',
  'TRBV7-3*01(L)'],
 'ATGGGCACCAGGCTCCTCTGCTGGGCAGCCCTGTGCCTCCTGGGGGCAGATCACACAGATTCTGGAGTCACACAAACCCCAAAGCACCTGATCACAGCAACTGGACAGCGAGTGACGCTGAGATGCTCCCCTAGGTCTGGAGACCTCTCTGTGTACTGGTACCAACAGAGCCTGGACCAGGGCCTCCAGTTCCTCATTCAGTATTATAATGGAGAAGAGAGAGCAAAAGGAAACATTCTTGAACGATTCTCCGCACAACAGTTCCCTGACTTGCACTCTGAACTAAACCTGAGCTCTCTGGAGCTGGGGGACTCAGCTTTGTATTTCTGTGCCAGCAGCGAGAACACCGCCAACACCGGGGAGCTGTTTTTTGGAGAAGGCTCTAGGCTGACCGTACTGGAGGACCTGAACAAGGTGTTCCCACCCGAGGTCGCTGTGTTTGAGCCATCAGAAGCAGAGATCTCCCACACCCAAAAGGCCACACTGGTGTGCCTGGCCACAGGCTTCTTCCCCGACCACGTGGAGCTGAGCTGGTGGGTGAATGGGAAGGAGGTGCACAGTGGGGTCAGCACGGACCCGCAGCCCCTCAAGGAGCAGCCCGCCCTCAATGACTCCAGATACTGCCTGAGCAGCCGCCTGAGGGTCTCGGCCACCTTCTGGCAGAACCCCCGCAACCACTTCCGCTGTCAAGTCCAGTTCTACGGGCTCTCGGAGAATGACGAGTGGACCCAGGATAGGGCCAAACCCGTCACCCAGATCGTCAGCGCCGAGGCCTGGGGTAGAGCAGACTGTGGCTTTACCTCGGTGTCCTACCAGCAAGGGGTCCTGTCTGCCACCATCCTCTATGAGATCCTGCTAGGGAAGGCCACCCTGTATGCTGTGCTGGTCAGCGCCCTTGTGTTGA

However the documentation mentions that for high throughput (as we must use) one should use [*thimble*](https://github.com/JamieHeather/stitchr/blob/main/docs/thimble.rst)